# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-12-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-12-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-13 04:29:15,33.939110,67.709953,37345,1354,26694,9297.0,Afghanistan,95.932678,3.625653
1,NaN,NaN,NaN,Albania,2020-08-13 04:29:15,41.153300,20.168300,6817,208,3552,3057.0,Albania,236.882341,3.051196
2,NaN,NaN,NaN,Algeria,2020-08-13 04:29:15,28.033900,1.659600,36699,1333,25627,9739.0,Algeria,83.690142,3.632252
3,NaN,NaN,NaN,Andorra,2020-08-13 04:29:15,42.506300,1.521800,977,53,855,69.0,Andorra,1264.479389,5.424770
4,NaN,NaN,NaN,Angola,2020-08-13 04:29:15,-11.202700,17.873900,1762,80,577,1105.0,Angola,5.361120,4.540295
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-13 04:29:15,17.060800,-61.796400,92,3,76,13.0,Antigua and Barbuda,93.946573,3.260870
6,NaN,NaN,NaN,Argentina,2020-08-13 04:29:15,-38.416100,-63.616700,268574,5213,187283,76078.0,Argentina,594.245785,1.940992
7,NaN,NaN,NaN,Armenia,2020-08-13 04:29:15,40.069100,45.038200,40794,806,33492,6496.0,Armenia,1376.671569,1.975781
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-13 04:29:15,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-13 04:29:15,-33.868800,151.209300,3927,51,2994,882.0,"New South Wales, Australia",48.373984,1.298701


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36747,36782,36829,36896,37015,37054,37054,37162,37269,37345
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5620,5750,5889,6016,6151,6275,6411,6536,6676,6817
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,31972,32504,33055,33626,34155,34693,35160,35712,36204,36699


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1288,1288,1294,1298,1307,1312,1312,1328,1344,1354
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,172,176,182,188,189,193,199,200,205,208
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1239,1248,1261,1273,1282,1293,1302,1312,1322,1333


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25669,25669,25742,25840,25903,25960,25960,26228,26415,26694
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3031,3031,3123,3155,3227,3268,3342,3379,3480,3552
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,21901,22375,22802,23238,23667,24083,24506,24920,25263,25627


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1024,1030,1030,1050,1065,1086,1169,1174,1162,1188
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3210,3263,3322,3380,3443,3502,3544,3651,3670,3710
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,563,563,564,565,569,573,575,576,579,581


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,20,20,21,21,21,21,21,21,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,23,23,23,23,23,24,25,25,29,29
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5,5,5,5,5,5,5,5,6,6


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-13 04:29:15,32.539527,-86.644082,1188,22,0,1166.0,"Autauga, Alabama, US",2126.402835,1.851852
668,1075.0,Lamar,Alabama,US,2020-08-13 04:29:15,33.779950,-88.096680,230,2,0,228.0,"Lamar, Alabama, US",1666.063021,0.869565
669,1077.0,Lauderdale,Alabama,US,2020-08-13 04:29:15,34.901719,-87.656247,1203,20,0,1183.0,"Lauderdale, Alabama, US",1297.328775,1.662510


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5197118,166026,1753760
Brazil,3164785,104201,2506228
India,2396637,47033,1695982


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5197118,166026,1753760,3277332,2020-08-13 04:29:15,37.909085,-91.336884
Brazil,3164785,104201,2506228,554356,2020-08-13 04:29:15,-12.669522,-48.480493
India,2396637,47033,1695982,653622,2020-08-13 04:29:15,23.088275,81.806127
Russia,900745,15231,708900,176614,2020-08-13 04:29:15,54.546312,62.120860
South Africa,568919,11010,432029,125880,2020-08-13 04:29:15,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,591376,10753,0
Florida,550901,8765,0
Texas,523977,9527,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,591376,10753,0,580623,2020-08-13 04:29:15,37.843962,-120.728594
Florida,550901,8765,0,542136,2020-08-13 04:29:15,28.940755,-82.700744
Texas,523977,9527,0,514450,2020-08-13 04:29:15,31.660643,-98.653069
New York,422703,32797,0,389906,2020-08-13 04:29:15,42.671593,-75.579694
Georgia,226153,4456,0,221697,2020-08-13 04:29:15,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,228729,23602,0
California,Los Angeles,214283,5112,0
Florida,Miami-Dade,139271,1939,0
Arizona,Maricopa,127188,2493,0
Illinois,Cook,113044,4934,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,228729,23602,0,205127,2020-08-13 04:29:15,40.767273,-73.971526,36061.0
California,Los Angeles,214283,5112,0,209171,2020-08-13 04:29:15,34.308284,-118.228241,6037.0
Florida,Miami-Dade,139271,1939,0,137332,2020-08-13 04:29:15,25.611236,-80.551706,12086.0
Arizona,Maricopa,127188,2493,0,124695,2020-08-13 04:29:15,33.348359,-112.491815,4013.0
Illinois,Cook,113044,4934,0,108110,2020-08-13 04:29:15,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,37162,6536,35712,963,1679,92,253868,40433,21713,22106,...,313392,1364,31304,26800,847,14510,10,1832,8210,4748
2020-08-11,37269,6676,36204,963,1735,92,260911,40593,22127,22245,...,314542,1385,31747,27938,866,14875,10,1831,8275,4818
2020-08-12,37345,6817,36699,977,1762,92,268574,40794,22358,22439,...,315581,1393,32654,29088,883,15184,10,1841,8501,4893


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,1328,200,1312,52,78,3,4764,796,331,723,...,46611,37,200,229,15,100,1,518,241,104
2020-08-11,1344,205,1322,52,80,3,5004,803,352,723,...,46714,37,204,238,16,104,1,523,241,104
2020-08-12,1354,208,1333,53,80,3,5213,806,361,724,...,46791,37,211,247,18,105,1,528,246,122


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,26228,3379,24920,839,569,76,108242,32616,12144,20010,...,1463,1146,22992,16930,399,8045,8,915,6802,1524
2020-08-11,26415,3480,25263,839,575,76,181389,33157,12395,20123,...,1464,1157,23704,19706,451,8181,8,915,7004,1524
2020-08-12,26694,3552,25627,855,577,76,187283,33492,12779,20268,...,1472,1163,25659,21042,409,8369,8,937,7233,1620


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1024,1030,1030,1050,1065,1086,1169,1174,1162,1188
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,198,203,205,212,217,220,222,229,230,230
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1071,1095,1111,1123,1139,1157,1167,1189,1190,1203
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,310,324,325,332,344,348,349,351,354,356
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2505,2519,2554,2588,2628,2660,2678,2700,2708,2724


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-10,103020,3774,0,187523,50028,574231,51020,50567,15634,49,...,123914,509581,44390,1462,547,100749,63647,7757,61061,3042
2020-08-11,103851,3819,0,188737,50411,578946,51422,50684,15699,49,...,124915,517700,44752,1472,576,101745,64151,7879,61785,3073
2020-08-12,104786,3879,0,189443,51096,591376,51737,50706,15765,49,...,126393,523977,45090,1478,639,102521,64702,8018,62263,3086


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-10        1174    3651     576  442    812     466    767    1807   
2020-08-11        1162    3670     579  444    816     481    770    1819   
2020-08-12        1188    3710     581  453    825     492    773    1828   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-10          844      275  ...     135      5       73       39   
2020-08-11          847      276  ...     135      5       73       39   
2020-08-12          848      279  ...     135      5       73       40   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-10            260   370   277          0       77      6  
2020-08-11            267   372   279          0       79      5  
2020-08-12            266   373   278          0       82      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-10,1797,26,0,4154,555,10476,1862,4444,591,0,...,1233,9230,345,58,9,2327,1697,141,998,28
2020-08-11,1847,26,0,4199,566,10523,1874,4444,591,0,...,1271,9222,349,58,9,2344,1716,147,1006,29
2020-08-12,1882,27,0,4347,572,10753,1875,4450,592,0,...,1289,9527,351,58,9,2352,1724,153,1011,29


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-10          21      25       5    5      4      11     36      17   
2020-08-11          22      29       6    5      5      11     36      18   
2020-08-12          22      29       6    5      5      11     36      19   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-10           38        8  ...       0      0        0        0   
2020-08-11           38        8  ...       0      0        0        0   
2020-08-12           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-10              0     0     0         27        0      0  
2020-08-11              0     0     0         28        0      0  
2020-08-12              0     0     0         28        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,0.002915,0.019498,0.015700,0.008377,0.004187,0.0,0.029895,0.000569,0.014768,0.003313,...,0.002617,0.008130,0.022706,0.038558,0.007134,0.021256,0.0,0.015521,0.015461,0.021295
2020-08-11,0.002879,0.021420,0.013777,0.000000,0.033353,0.0,0.027743,0.003957,0.019067,0.006288,...,0.003670,0.015396,0.014152,0.042463,0.022432,0.025155,0.0,-0.000546,0.007917,0.014743
2020-08-12,0.002039,0.021120,0.013673,0.014538,0.015562,0.0,0.029370,0.004952,0.010440,0.008721,...,0.003303,0.005776,0.028570,0.041163,0.019630,0.020773,0.0,0.005461,0.027311,0.015567


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,0.012195,0.005025,0.007680,0.000000,0.040000,0.0,0.034303,0.006321,0.057508,0.002774,...,-0.001029,0.0,0.030928,0.026906,0.153846,0.030928,0.0,0.005825,0.025532,0.000000
2020-08-11,0.012048,0.025000,0.007622,0.000000,0.025641,0.0,0.050378,0.008794,0.063444,0.000000,...,0.002210,0.0,0.020000,0.039301,0.066667,0.040000,0.0,0.009653,0.000000,0.000000
2020-08-12,0.007440,0.014634,0.008321,0.019231,0.000000,0.0,0.041767,0.003736,0.025568,0.001383,...,0.001648,0.0,0.034314,0.037815,0.125000,0.009615,0.0,0.009560,0.020747,0.173077


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,0.010324,0.011071,0.016894,0.00000,0.003527,0.0,0.000000,0.002952,0.022567,0.004367,...,0.008270,0.018667,0.043100,0.267595,0.010127,0.012587,0.0,0.002191,0.015527,0.060543
2020-08-11,0.007130,0.029891,0.013764,0.00000,0.010545,0.0,0.675773,0.016587,0.020669,0.005647,...,0.000684,0.009599,0.030967,0.163969,0.130326,0.016905,0.0,0.000000,0.029697,0.000000
2020-08-12,0.010562,0.020690,0.014408,0.01907,0.003478,0.0,0.032494,0.010103,0.030980,0.007206,...,0.005464,0.005186,0.082476,0.067797,-0.093126,0.022980,0.0,0.024044,0.032696,0.062992


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-10,0.016638,0.016977,NaN,0.003210,0.013061,0.025878,0.007464,0.004909,0.003788,0.0,...,0.009795,0.012423,0.005960,0.002056,0.035985,0.006624,0.009117,0.008057,0.008373,-0.002623
2020-08-11,0.008066,0.011924,NaN,0.006474,0.007656,0.008211,0.007879,0.002314,0.004158,0.0,...,0.008078,0.015933,0.008155,0.006840,0.053016,0.009886,0.007919,0.015728,0.011857,0.010191
2020-08-12,0.009003,0.015711,NaN,0.003741,0.013588,0.021470,0.006126,0.000434,0.004204,0.0,...,0.011832,0.012125,0.007553,0.004076,0.109375,0.007627,0.008589,0.017642,0.007737,0.004230


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-10      0.004277  0.030192  0.001739  0.009132  0.015000  0.004310   
2020-08-11     -0.010221  0.005204  0.005208  0.004525  0.004926  0.032189   
2020-08-12      0.022375  0.010899  0.003454  0.020270  0.011029  0.022869   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-10      0.003927  0.012892  0.003567  0.007326  ...  0.007463    0.0   
2020-08-11      0.003911  0.006641  0.003555  0.003636  ...  0.000000    0.0   
2020-08-12      0.003896  0.004948  0.001181  0.010870  ...  0.000000    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-10     -0.013514 -0.025000   0.003861 -0.010695 -0.003597        NaN   
2020-08-11      0.000000  0.000000   0.026923  0.005405  0.007220        NaN   
2020-08-12      0.000000  0.025641  -0.003745  0.002688 -0.003584        NaN   

Province_State                      
Admin2          Washakie    Weston  
2020-08-10      0.000000  0.200000  
2020-08-11      0.025974 -0.166667  
2020-08-12      0.037975  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-10,0.016403,0.000000,NaN,0.000964,0.020221,0.009540,0.002153,0.000676,0.000000,NaN,...,0.008177,0.007972,0.026786,0.0,0.0,0.000430,0.005332,0.014388,0.000000,0.000000
2020-08-11,0.027824,0.000000,NaN,0.010833,0.019820,0.004486,0.006445,0.000000,0.000000,NaN,...,0.030819,-0.000867,0.011594,0.0,0.0,0.007306,0.011196,0.042553,0.008016,0.035714
2020-08-12,0.018950,0.038462,NaN,0.035246,0.010601,0.021857,0.000534,0.001350,0.001692,NaN,...,0.014162,0.033073,0.005731,0.0,0.0,0.003413,0.004662,0.040816,0.004970,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-08-10      0.000000    0.00     0.0  0.0   0.00     0.0    0.0  0.214286   
2020-08-11      0.047619    0.16     0.2  0.0   0.25     0.0    0.0  0.058824   
2020-08-12      0.000000    0.00     0.0  0.0   0.00     0.0    0.0  0.055556   

Province_State                     ... Wyoming                           \
Admin2         Chambers  Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-10          0.0  0.142857  ...     NaN    NaN      NaN      NaN   
2020-08-11          0.0  0.000000  ...     NaN    NaN      NaN      NaN   
2020-08-12          0.0  0.000000  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-10            NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-11            NaN   NaN   NaN   0.037037      NaN    NaN  
2020-08-12            NaN   NaN   NaN   0.000000      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,0.001883,0.020478,0.015235,0.005155,0.026407,0.000052,0.027146,0.002939,0.016193,0.004152,...,0.002874,0.009140,0.025150,0.036750,0.022548,0.020493,9.868649e-17,0.010036,0.024960,0.021424
2020-08-11,0.002381,0.020949,0.014506,0.002578,0.029880,0.000026,0.027444,0.003448,0.017630,0.005220,...,0.003272,0.012268,0.019651,0.039606,0.022490,0.022824,4.934325e-17,0.004745,0.016439,0.018083
2020-08-12,0.002210,0.021035,0.014089,0.008558,0.022721,0.000013,0.028407,0.004200,0.014035,0.006970,...,0.003287,0.009022,0.024110,0.040384,0.021060,0.021798,2.467162e-17,0.005103,0.021875,0.016825


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,0.007202,0.015256,0.007670,2.721135e-19,0.048647,3.032775e-36,0.029462,0.007026,0.057218,0.001660,...,-0.000106,0.000274,0.033094,0.031164,0.161226,0.023919,0.0,0.005078,0.058999,0.020379
2020-08-11,0.009625,0.020128,0.007646,1.360567e-19,0.037144,1.516388e-36,0.039920,0.007910,0.060331,0.000830,...,0.001052,0.000137,0.026547,0.035233,0.113923,0.031960,0.0,0.007366,0.029500,0.010189
2020-08-12,0.008533,0.017381,0.007983,9.615385e-03,0.018572,7.581939e-37,0.040843,0.005823,0.042950,0.001106,...,0.001350,0.000068,0.030430,0.036524,0.119463,0.020788,0.0,0.008463,0.025123,0.091633


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-10,0.005780,0.015083,0.017426,0.001051,0.012308,0.000681,0.010011,0.006008,0.018163,0.005124,...,0.004694,0.016061,0.044739,0.144236,0.006665,0.028855,5.782412e-19,0.003674,0.030215,0.046869
2020-08-11,0.006455,0.022487,0.015595,0.000526,0.011426,0.000341,0.342892,0.011298,0.019416,0.005385,...,0.002689,0.012830,0.037853,0.154103,0.068496,0.022880,2.891206e-19,0.001837,0.029956,0.023435
2020-08-12,0.008508,0.021588,0.015002,0.009798,0.007452,0.000170,0.187693,0.010700,0.025198,0.006296,...,0.004077,0.009008,0.060164,0.110950,-0.012315,0.022930,1.445603e-19,0.012940,0.031326,0.043213


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-10,0.015726,0.019877,NaN,0.004400,0.013893,0.018702,0.007667,0.002627,0.004255,1.908151e-45,...,0.013593,0.012744,0.007624,0.002825,0.024619,0.009151,0.010143,0.012725,0.011233,0.003957
2020-08-11,0.011896,0.015900,NaN,0.005437,0.010774,0.013457,0.007773,0.002470,0.004206,9.540756e-46,...,0.010836,0.014339,0.007889,0.004833,0.038818,0.009519,0.009031,0.014226,0.011545,0.007074
2020-08-12,0.010450,0.015806,NaN,0.004589,0.012181,0.017463,0.006949,0.001452,0.004205,4.770378e-46,...,0.011334,0.013232,0.007721,0.004454,0.074096,0.008573,0.008810,0.015934,0.009641,0.005652


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-10      0.025380  0.022493  0.003201  0.017865  0.015720  0.006345   
2020-08-11      0.007579  0.013848  0.004205  0.011195  0.010323  0.019267   
2020-08-12      0.014977  0.012374  0.003830  0.015733  0.010676  0.021068   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-10      0.004199  0.016744  0.003143  0.012674  ...  0.011462   
2020-08-11      0.004055  0.011693  0.003349  0.008155  ...  0.005731   
2020-08-12      0.003976  0.008320  0.002265  0.009512  ...  0.002866   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-08-10      2.384188e-07  0.009309  0.000435   0.005193 -0.001685   
2020-08-11      1.192094e-07  0.004654  0.000218   0.016058  0.001860   
2020-08-12      5.960471e-08  0.002327  0.012929   0.006156  0.002274   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-08-10      0.003000       -1.0  0.039945  0.100008  
2020-08-11      0.005110       -1.0  0.032960 -0.033330  
2020-08-12      0.000763       -1.0  0.035467 -0.016665  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-10,0.013112,0.005221,NaN,0.005326,0.019805,0.010454,0.001444,0.000399,0.000986,NaN,...,0.009532,0.013774,0.015920,0.000553,0.003924,0.002081,0.005315,0.028421,0.002561,0.002468
2020-08-11,0.020468,0.002611,NaN,0.008080,0.019813,0.007470,0.003944,0.000200,0.000493,NaN,...,0.020176,0.006454,0.013757,0.000276,0.001962,0.004693,0.008256,0.035487,0.005288,0.019091
2020-08-12,0.019709,0.020536,NaN,0.021663,0.015207,0.014664,0.002239,0.000775,0.001093,NaN,...,0.017169,0.019763,0.009744,0.000138,0.000981,0.004053,0.006459,0.038152,0.005129,0.009545


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-10      0.000781  0.016168  0.000061  0.037435  0.041911   
2020-08-11      0.024200  0.088084  0.100031  0.018717  0.145955   
2020-08-12      0.012100  0.044042  0.050015  0.009359  0.072978   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-08-10      9.536782e-08  0.003572  0.130422  1.192504e-06  0.071429  ...   
2020-08-11      4.768391e-08  0.001786  0.094623  5.962520e-07  0.035714  ...   
2020-08-12      2.384196e-08  0.000893  0.075089  2.981260e-07  0.017857  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-10         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-11         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-12         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-10       0.002563      NaN    NaN  
2020-08-11       0.019800      NaN    NaN  
2020-08-12       0.009900      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200812,AK,4655,287927,NaN,39.0,NaN,NaN,NaN,3.0,...,292582,1,0,04d3f681722d466f15660e1ded53888693484520,0,0,0,0,0,NaN
1,20200812,AL,104786,690985,NaN,1372.0,12292.0,NaN,1282.0,NaN,...,795771,35,222,3b8e75d37adb9ba3951ec8b2aae85ba1fa9b02f1,0,0,0,0,0,NaN
2,20200812,AR,51114,522457,NaN,486.0,3472.0,NaN,NaN,113.0,...,573571,0,71,614656008210925b859900619d0426d793e79029,0,0,0,0,0,NaN
3,20200812,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,f6988932e32e01718e47c8647131e3311e70d481,0,0,0,0,0,NaN
4,20200812,AZ,189443,854785,NaN,1469.0,19821.0,519.0,NaN,328.0,...,1044228,148,529,83481965d3dca76744e02ade2b8ec71c6fec31b7,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200812,AK,4655.0,287927.0,NaN,39.0,NaN,NaN,NaN,3.0,...,292582,1,0,04d3f681722d466f15660e1ded53888693484520,0,0,0,0,0,NaN
1,20200812,AL,104786.0,690985.0,NaN,1372.0,12292.0,NaN,1282.0,NaN,...,795771,35,222,3b8e75d37adb9ba3951ec8b2aae85ba1fa9b02f1,0,0,0,0,0,NaN
2,20200812,AR,51114.0,522457.0,NaN,486.0,3472.0,NaN,NaN,113.0,...,573571,0,71,614656008210925b859900619d0426d793e79029,0,0,0,0,0,NaN
3,20200812,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,f6988932e32e01718e47c8647131e3311e70d481,0,0,0,0,0,NaN
4,20200812,AZ,189443.0,854785.0,NaN,1469.0,19821.0,519.0,NaN,328.0,...,1044228,148,529,83481965d3dca76744e02ade2b8ec71c6fec31b7,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-12,AK,4655,287927,NaN,39.0,NaN,NaN,NaN,3.0,NaN,...,292582,1,0,04d3f681722d466f15660e1ded53888693484520,0,0,0,0,0,NaN
2020-08-12,AL,104786,690985,NaN,1372.0,12292.0,NaN,1282.0,NaN,700.0,...,795771,35,222,3b8e75d37adb9ba3951ec8b2aae85ba1fa9b02f1,0,0,0,0,0,NaN
2020-08-12,AR,51114,522457,NaN,486.0,3472.0,NaN,NaN,113.0,455.0,...,573571,0,71,614656008210925b859900619d0426d793e79029,0,0,0,0,0,NaN
2020-08-12,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,f6988932e32e01718e47c8647131e3311e70d481,0,0,0,0,0,NaN
2020-08-12,AZ,189443,854785,NaN,1469.0,19821.0,519.0,NaN,328.0,NaN,...,1044228,148,529,83481965d3dca76744e02ade2b8ec71c6fec31b7,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-12,AK,4655.0,287927.0,NaN,39.0,NaN,NaN,NaN,3.0,NaN,...,292582,1,0,04d3f681722d466f15660e1ded53888693484520,0,0,0,0,0,NaN
2020-08-12,AL,104786.0,690985.0,NaN,1372.0,12292.0,NaN,1282.0,NaN,700.0,...,795771,35,222,3b8e75d37adb9ba3951ec8b2aae85ba1fa9b02f1,0,0,0,0,0,NaN
2020-08-12,AR,51114.0,522457.0,NaN,486.0,3472.0,NaN,NaN,113.0,455.0,...,573571,0,71,614656008210925b859900619d0426d793e79029,0,0,0,0,0,NaN
2020-08-12,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,f6988932e32e01718e47c8647131e3311e70d481,0,0,0,0,0,NaN
2020-08-12,AZ,189443.0,854785.0,NaN,1469.0,19821.0,519.0,NaN,328.0,NaN,...,1044228,148,529,83481965d3dca76744e02ade2b8ec71c6fec31b7,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE